In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
from torch import cuda, bfloat16

In [5]:
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

cuda:0


In [6]:
import transformers

In [7]:
bnb_config = transformers.BitsAndBytesConfig(
# bnb_config = BitsAndBytesConfig(
    #load_in_4bit=True,
    load_in_4bit_fp32_cpu_offload=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [8]:
#model_name = 'tiiuae/falcon-40b-instruct'
model_name = 'tiiuae/falcon-7b-instruct'

In [9]:
%%time
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
    offload_folder="/mnt/d/AAA/VsCode/offload/falcon-7b-instruct" # Disk path to load the model when GPU, CPU runs out of memo
    #device_map=device_map 
)
model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0
CPU times: user 13.3 s, sys: 12.8 s, total: 26.1 s
Wall time: 20 s


In [10]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [11]:
from transformers import StoppingCriteria, StoppingCriteriaList

# we create a list of stopping criteria
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [
        ['Human', ':'], ['AI', ':']
    ]
]

stop_token_ids

[[23431, 37], [17362, 37]]

In [12]:
# We need to convert these into `LongTensor` objects:
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([23431,    37], device='cuda:0'),
 tensor([17362,    37], device='cuda:0')]

In [13]:
# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [14]:
# Now we're ready to initialize the HF pipeline. There are a few additional parameters that we must define here. Comments explaining these have been included in the code.
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [15]:
%%time
# Confirm this is working
res = generate_text("Explain to me the difference between nuclear fission and fusion.")
print(res[0]["generated_text"])

/home/gourab_deb/Projects/VsCode/langchain-experiments/pinecone-io-examples/generation/llm-field-guide/falcon-40b/env-torch-cuda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Explain to me the difference between nuclear fission and fusion.
Nuclear fission is a nuclear reaction in which the nucleus of an atom splits into two smaller nuclei, releasing a large amount of energy in the form of radiation and kinetic energy of the fragments. Fusion, on the other hand, is a nuclear reaction in which two lighter atomic nuclei combine to form a heavier atomic nucleus, releasing a large amount of energy in the form of radiation and kinetic energy of the resulting nucleus. In both reactions, the binding energy of the nucleus is released, allowing for the release of a significant amount of energy.
CPU times: user 6min 38s, sys: 2min 22s, total: 9min 1s
Wall time: 9min 26s


In [16]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

In [17]:
# template for an instruction with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}"
)

In [18]:
%%time
llm = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=llm, prompt=prompt)

CPU times: user 262 µs, sys: 98 µs, total: 360 µs
Wall time: 387 µs


In [20]:
%%time
print(llm_chain.predict(
    instruction="Explain to me the difference between nuclear fission and fusion."
).lstrip())

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Nuclear fission is a nuclear reaction in which the nucleus of an atom splits into two smaller nuclei, releasing a large amount of energy in the form of radiation and kinetic energy of the fragments. Fusion, on the other hand, is a nuclear reaction in which two lighter atomic nuclei combine to form a heavier atomic nucleus, releasing a large amount of energy in the form of radiation and kinetic energy of the resulting nucleus. In both reactions, the binding energy of the nucleus is released, allowing for the release of a significant amount of energy.
CPU times: user 6min 32s, sys: 2min 17s, total: 8min 50s
Wall time: 9min 14s
